In [1]:
! git clone https://github.com/someshsingh22/Sherlocked

Cloning into 'Sherlocked'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 17 (delta 1), reused 16 (delta 0), pack-reused 0
Unpacking objects: 100% (17/17), done.


In [2]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import time
import csv
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers import CuDNNLSTM,Bidirectional
from keras.layers.wrappers import TimeDistributed
import re

Using TensorFlow backend.


In [0]:
def cleaner(file_path):
  text = open(file_path).read()
  text=re.sub(r'\n+', '\n',text).strip()
  text=re.sub(' +', ' ',text).strip()
  return text

In [0]:
file_path="./Sherlocked/Dataset/cano.txt"
Data=cleaner(file_path)

In [0]:
def generate_text(model, length, vocab_size, ix_to_char):
	# starting with random character
	ix = [np.random.randint(vocab_size)]
	y_char = [ix_to_char[ix[-1]]]
	X = np.zeros((1, length, vocab_size))
	for i in range(length):
		# appending the last predicted character to sequence
		X[0, i, :][ix[-1]] = 1
		print(ix_to_char[ix[-1]], end="")
		ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
		y_char.append(ix_to_char[ix[-1]])
	return ('').join(y_char)

# method for preparing the training data
def load_data(data, seq_length):
	chars = list(set(data))
	VOCAB_SIZE = len(chars)

	print('Data length: {} characters'.format(len(data)))
	print('Vocabulary size: {} characters'.format(VOCAB_SIZE))

	ix_to_char = {ix:char for ix, char in enumerate(chars)}
	char_to_ix = {char:ix for ix, char in enumerate(chars)}

	X = np.zeros((len(data)//seq_length, seq_length, VOCAB_SIZE))
	y = np.zeros((len(data)//seq_length, seq_length, VOCAB_SIZE))
	for i in range(0, len(data)//seq_length):
		X_sequence = data[i*seq_length:(i+1)*seq_length]
		X_sequence_ix = [char_to_ix[value] for value in X_sequence]
		input_sequence = np.zeros((seq_length, VOCAB_SIZE))
		for j in range(seq_length):
			input_sequence[j][X_sequence_ix[j]] = 1.
			X[i] = input_sequence

		y_sequence = data[i*seq_length+1:(i+1)*seq_length+1]
		y_sequence_ix = [char_to_ix[value] for value in y_sequence]
		target_sequence = np.zeros((seq_length, VOCAB_SIZE))
		for j in range(seq_length):
			target_sequence[j][y_sequence_ix[j]] = 1.
			y[i] = target_sequence
	return X, y, VOCAB_SIZE, ix_to_char

In [0]:
BATCH_SIZE=64
HIDDEN_DIM=700
SEQ_LENGTH=60

In [0]:
X, y, VOCAB_SIZE, ix_to_char = load_data(Data, SEQ_LENGTH)

Data length: 3604668 characters
Vocabulary size: 97 characters


In [0]:
model = Sequential()
model.add(Bidirectional(CuDNNLSTM(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(CuDNNLSTM(HIDDEN_DIM, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(CuDNNLSTM(HIDDEN_DIM, return_sequences=True)))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [0]:
model.fit(X, y, batch_size=BATCH_SIZE, verbose=1, epochs=10)

In [0]:
 generate_text(model, 50, VOCAB_SIZE, ix_to_char)